**Задача**: Вывести таблицу с продажами по годам, по странам, в сегменах продукта M2 и M4. При этом продажи по Казахстану должны быть только в 2020м году, все остальные продажи KZ не должны отображаться. 

На выходе должны получить следующие столбцы:
* `Year`
* `Country`
* `segment`
* `Sales_Kg`

In [2]:
import pandas as pd

In [3]:
# импортируем 1 таблицу sales (уже готова после задачи с SQL)
df_sales = pd.read_csv(r'C:\Users\User\Desktop\data_analytics\Job tests\BND\csv\sales.csv')
df_sales 

,sales_date,client_key,product_key,sales_kg
0,2019-01-01,1,201,101.2
1,2019-08-01,2,202,250.6
2,2020-01-01,3,203,145.6
3,2020-04-03,4,204,111.1
4,2020-06-01,5,205,311.1
5,2020-06-05,6,206,200.6
6,2019-09-15,7,207,135.6


In [4]:
# импортируем 2 таблицу client (уже готова после задачи с SQL)
df_client = pd.read_csv(r'C:\Users\User\Desktop\data_analytics\Job tests\BND\csv\client.csv')
df_client.drop('name', axis = 1, inplace = True)
df_client

,key,country
0,1,RUS
1,2,KZ
2,3,KZ
3,4,KZ
4,5,BY
5,6,AM
6,7,AZ


In [5]:
# импортируем 3 таблицу product (уже готова после задачи с SQL)
df_product = pd.read_csv(r'C:\Users\User\Desktop\data_analytics\Job tests\BND\csv\product.csv')
df_product

,key,name,segment
0,201,Chocolate,M1
1,202,Gum,M2
2,203,Water,M3
3,204,Fish,M4
4,205,Mint,M2
5,206,Juice,M4
6,207,Meat,M2


In [6]:
df_sales['year'] = pd.DatetimeIndex(df_sales['sales_date']).year #выводим из даты только год в новую колонку year
df_sales.drop('sales_date', axis = 1, inplace = True) #следом убираем колонку с первичной датой
df_sales

,client_key,product_key,sales_kg,year
0,1,201,101.2,2019
1,2,202,250.6,2019
2,3,203,145.6,2020
3,4,204,111.1,2020
4,5,205,311.1,2020
5,6,206,200.6,2020
6,7,207,135.6,2019


In [7]:
df_client

,key,country
0,1,RUS
1,2,KZ
2,3,KZ
3,4,KZ
4,5,BY
5,6,AM
6,7,AZ


In [9]:
df_product.drop('name', axis = 1, inplace = True) #убираем колону с именами
df_product

,key,segment
0,201,M1
1,202,M2
2,203,M3
3,204,M4
4,205,M2
5,206,M4
6,207,M2


In [10]:
# данный скрипт позволяет скреплять таблицы, при этом не указывая джойны
concatenated_dataframes = pd.concat(
    [
        df_sales.reset_index(drop=True),
        df_client.reset_index(drop=True),
        df_product.reset_index(drop=True)
    ],
    axis=1,
    ignore_index=True,
)

concatenated_dataframes_columns = [
    list(df_sales.columns),
    list(df_client.columns),
    list( df_product.columns)
]
    
flatten = lambda nested_lists: [item for sublist in nested_lists for item in sublist]

concatenated_dataframes.columns = flatten(concatenated_dataframes_columns)

In [11]:
# скрипт выше порождает дупликатные колонки, делаем новую переменную с нужными нам фичами
concatenated_dataframes_final = concatenated_dataframes[(concatenated_dataframes['segment'] == 'M2')|(concatenated_dataframes['segment'] == 'M4')]
concatenated_dataframes_final = concatenated_dataframes_final[['year','country','segment','sales_kg' ]]
concatenated_dataframes_final


,year,country,segment,sales_kg
1,2019,KZ,M2,250.6
3,2020,KZ,M4,111.1
4,2020,BY,M2,311.1
5,2020,AM,M4,200.6
6,2019,AZ,M2,135.6


Теперь, когда мы получили все в сегментах М2 и М4, у нас все еще есть продажи из КЗ, которые были не в 2020 году, код ниже с этим разберется :)

In [13]:
# задаем условие, что если из КЗ и год не 2020, то убираем ряд из таблицы
for index, row in concatenated_dataframes_final.iterrows():
    if row['country'] == 'KZ' and row['year'] != 2020:
        concatenated_dataframes_final.drop(index, inplace=True)
result = concatenated_dataframes_final
# А вот и результат!
print(result.to_string(index=False))

 year country segment  sales_kg
 2020      KZ      M4     111.1
 2020      BY      M2     311.1
 2020      AM      M4     200.6
 2019      AZ      M2     135.6
